In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
from fake_useragent import UserAgent
import pandas as pd

In [2]:
def get_soup_retry(url):
    '''This is to evade annoying bot interceptors'''
    ua= UserAgent()
    uag_rand= ua.random
    header={
        'user-agent': uag_rand,
        "accept-language": 'en-US'
    }
    isCaptcha = True
    while isCaptcha:
            page = requests.get(url, headers=header)
            assert page.status_code==200
            soup=BeautifulSoup(page.content, 'lxml')
            if 'captcha' in str(soup):
                uag_rand = ua.random
                print(f'\rbot detected..t=retrying..use new ua: {uag_rand}', end='', flush=True)
            else: 
                print('bot bypassed')
                return soup

def search_keyword(kw):
    count_page=0
    count_asin=0
    while True:
        count_page +=1
        url= f"https://www.amazon.ca/s?k={kw}&page={count_page}"
        print(f'Getting page: {count_page}|{url}')
        soup=get_soup_retry(url)
        result= soup.find('div', attrs={'class':'s-main-slot s-result-list s-search-results sg-row'}).find_all("div", attrs={'data-component-type': 's-search-result'})
        
        for ids in result:
            count_asin +=1
            asin = ids['data-asin']
            url=f'https://www.amazon.com/dp/{asin}'
            '''see below funtions for getting all the details on each item then write it to a df and to csv'''
            list_result = get_details(url)
            df=pd.DataFrame(list_result)
            df.to_csv(f'result_amazon.csv', index=False)
            
            
            print(f'{count_asin}.{url}')
            
        last_page = soup.find('li', attrs={'class':'a-disabled a-last'})
        if not last_page:
            pass
        else:
            break

result=[]

def get_details(url):
    soup = get_soup_retry(url)
    today = datetime.date.today()
    title = soup.find('span', attrs={'id':'productTitle'}).text.strip()
    try:
        price = soup.find('span', attrs={'class':"a-offscreen"}).text.strip() 
    except AttributeError:
        price=''
    try:
        review_count = soup.find('span', attrs={"id": 'acrCustomerReviewText'}).text.strip()
    except AttributeError:
        review_count='' 
    try:
        feature= soup.find("div", attrs={'id':'feature-bullets'}).find('ul', attrs={'class':'a-unordered-list a-vertical a-spacing-mini'}).find_all(('li'))
    
        sv_feature=[]
        for li in feature:
            text=li.find('span', attrs={'class':'a-list-item'})
            f=str(text.string).strip()
            if "None" in f:
                pass
            else:
                sv_feature.append(f)
    except AttributeError:
        sv_feature =''
    try:
        stock=soup.find('div', attrs={'id':"availability"}).find('span').text.strip()
    except AttributeError:
        stock ='' 
    try:
        description=soup.find('div', attrs={'id':"productDescription"}).text.strip()
    except AttributeError:
        description ='' 
        
    print(today)
    print(title)
    print(price)
    print(review_count)
    print(sv_feature)
    print(stock)
    print(description)
    
    goal={
        'datetime': today,
        'title':title,
        'price':price,
        'review_count': review_count,
        'features': sv_feature,
        'stock': stock,
        #'description': description
    }
    print(goal)
    result.append(goal)
    return result

In [5]:
search_keyword('amd+graphics+card+6800xt+6800xt+NVIDIA+3090+3080')

Getting page: 1|https://www.amazon.ca/s?k=amd+graphics+card+6800xt+6800xt+NVIDIA+3090+3080&page=1
bot bypassed
bot bypassed
2022-02-25
MSI Gaming Radeon RX 6800 XT 16GB GDRR6 256-Bit HDMI/DP 2285 MHz RDNA 2 Architecture OC Graphics Card (RX 6800 XT Gaming X Trio 16G)
$1,559.99
47 ratings
['Chipset: Radeon RX 6800 XT', 'Video Memory: 16GB GDDR6', 'Memory Interface: 256-bit', 'Output: DisplayPort x 3 (v1.4) / HDMI 2.1 x 1', 'Digital maximum resolution - 7680 x 4320']
Only 3 left in stock - order soon.
Get the ultimate game changer. AMD Radeon RX 6800 XT graphics card features breakthrough AMD RDNA 2 architecture. Now you can game in 4K with ultra-smooth frame rates and at max settings. Never again compromise on resolution to enjoy fluid, high-refresh-rate gaming. Experience a new level of immersion with the AMD Radeon RX 6800 XT graphics card.
{'datetime': datetime.date(2022, 2, 25), 'title': 'MSI Gaming Radeon RX 6800 XT 16GB GDRR6 256-Bit HDMI/DP 2285 MHz RDNA 2 Architecture OC Graphic

bot detected..t=retrying..use new ua: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36ari/537.36bot bypassed
2022-02-25
ZOTAC Gaming GeForce RTX™ 3070 Ti AMP Holo 8GB GDDR6X 256-bit 19 Gbps PCIE 4.0 Gaming Graphics Card, HoloBlack, IceStorm 2.0 Advanced Cooling, Spectra 2.0 RGB Lighting, ZT-A30710F-10P
$1,049.99
116 ratings
['NVIDIA Ampere architecture, 2nd Gen Ray Tracing Cores, 3rd Gen Tensor Cores', '8GB 256-bit GDDR6X, 19 Gbps, PCIE 4.0; Boost Clock 1830 MHz', 'HoloBlack, SPECTRA 2.0 RGB Lighting, IceStorm 2.0 Advanced Cooling, FREEZE Fan Stop, Active Fan Control, Lighted Metal Backplate', '8K Ready, 4 Display Ready, HDCP 2.3, VR Ready', '3 x DisplayPort 1.4a, 1 x HDMI 2.1, DirectX 12 Ultimate, Vulkan RT API, OpenGL 4.6']


{'datetime': datetime.date(2022, 2, 25), 'title': 'ZOTAC Gaming GeForce RTX™ 3070 Ti AMP Holo 8GB GDDR6X 256-bit 19 Gbps PCIE 4.0 Gaming Graphics Card, HoloBlack, IceStorm 2.0 Advanced Cooling, Spectra 

bot bypassed
2022-02-25
ASUS ROG Strix NVIDIA GeForce RTX 3060 V2 OC Edition Gaming Graphics Card (PCIe 4.0, 12GB GDDR6, HDMI 2.1, DisplayPort 1.4a, Axial-tech Fan Design, 2.7-Slot, Super Alloy Power II, GPU Tweak II)
$1,299.99
66 ratings
['NVIDIA Ampere Streaming Multiprocessors: The building blocks for the world’s fastest, most efficient GPU, the all-new Ampere SM brings 2X the FP32 throughput and improved power efficiency.', '2nd Generation RT Cores: Experience 2X the throughput of 1st gen RT Cores, plus concurrent RT and shading for a whole new level of ray tracing performance.', '3rd Generation Tensor Cores: Get up to 2X the throughput with structural sparsity and advanced AI algorithms such as DLSS. Now with support for up to 8K resolution, these cores deliver a massive boost in game performance and all-new AI capabilities.', 'OC mode: Boost clock 1912 MHz (OC mode)/ 1882 MHz (Gaming mode)', 'Axial-Tech Fan Design has been tuned up with more fan blades and a reversed rotational d

bot detected..t=retrying..use new ua: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.3636 Mozilla/5.0 (iPad; U; CPU OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B334b Safari/531.21.10

AssertionError: 

In [23]:
    '''
   Could be added below description in fun tion above. 
    
    dt_img = soup.select('#imageBlock_feature_div> script:nth-child(2)')
    try:
        script_text = dt_img[0].text
        json_str=re.search('{(.+)}', script_text)[0].replace("\'",'"').replace("null",'"null"')
        json_obj=json.loads(json_str)
        images_url=[]
        for i in json_obj['initial']:
            images_hires=i["hiRes"]
            images_large=i["large"]
            if images_hires is None:
                images_url.append(images_large)
            else:
                images_url.append(images_hires)
    except IndexErrorError:
        images_url=''
        

and add:
print(images_url)

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.